In this notebook, we're going to work with an extended version of the "Heatlh Care for All" dataset will lots of new columns. We will start reviewing the manual data cleaning process for numerical, and categorical columns for some columns. Later, during the week we will cover automated ways to select the best subsets of columns to be used in our analysis.

# 7.01


# Loading libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Reading data

In [ ]:
data = pd.read_csv('learningSet.csv')

In [ ]:
print(data.shape)
data.head()

Column "TARGET_B" encodes if the customer answered or not the call

In [ ]:
data['TARGET_B'].value_counts()

Column "TARGET_D" provide the donation amount.

In [ ]:
data['TARGET_D'].describe()

In [ ]:
data[data['TARGET_B']>0]['TARGET_D'].describe()

To solve this problem we will first build a classification model to predict who will more likely respond and then for those respondents, we will build a regression model to predict the donation amount.

Then we can use the cost matrix to calculate the total benefit from the donations (remember that the company has to do a mailing champaing and each envelope costs some money).

Some of the challenges with the dataset are as follows:

* Large number of features: The data set has over 450 features. Hence selecting the right features for the model is very critical and at the same time it is not easy as the same traditional ways of removing features is not effective given the large number of features. Apart from feature selection, feature extraction (creating your own features using the existing features) is also not easy in this case.

* Sparsity of the dataset: There are a lot of features with a large number of null values.

* Data imbalance: For developing a classification, there is a huge imbalance in the training dataset with only approximately 5000 values for one category as compared to cover 95,000 instances for the other category.

# Review data cleaning process

There are a lot of columns that have a very high percentage of null values. It is a highly sparse dataset.
We can decide on a threshold and then remove those variables. There is no rule of thumb to decide on this threshold value.
Sometimes it can as low as 25%-30%. And sometimes in some data sets you might find that even though there are more than 50% missing values in a column, you might have to include that variable in your analysis.
A lot of it depends on the business context as well. In this case we will take this threshold to be 25% and then check the definitions of the columns filtered, to see if there is any column that we might want to keep.

In [ ]:
data.shape[0]

In [ ]:
len(data)

In [ ]:
nulls_percent_df = pd.DataFrame(data.isna().sum()/len(data)).reset_index() # to calculate percentage missing values
nulls_percent_df.columns = ['column_name', 'nulls_percentage']
nulls_percent_df

Which columns have NA's?

In [ ]:
nulls_percent_df[nulls_percent_df['nulls_percentage']!=0] # to calculate which rows have missing values

In [ ]:
columns_above_threshold = nulls_percent_df[nulls_percent_df['nulls_percentage']>0.25] #j filtering the columns with above threshold
columns_above_threshold['column_name']

In [ ]:
drop_columns_list = list(columns_above_threshold['column_name'])
print(drop_columns_list)

From the list above that includes the columns that have over 25% null values, you discussed with your manager you were told that the following columns are important -> **wealth1, wealth2**
We will remove these variables from the above list
RDATE3, RAMNT_3 are important but they have too many null values

# Activity

Create a function that takes a dataframe as an input and a percentual threshold (default value = 0.25) and returns a list of columns with null values greater than the specified threshold

In [ ]:
def remove_cols_with_na(df: pd.DataFrame, thr = 0.25) -> list:
    nulls_percent_df = pd.DataFrame(df. isna().sum()/len(data)) .reset_index()
    nulls_percent_df.columns = ['column_name', 'nulls_percentage']
    return list(nulls_percent_df [nulls_percent_df['nulls_percentage'] > thr]['column_name'].values)

In [ ]:
# End of the activity

# More data cleaning

In [ ]:
data.head()

We can see that there are a lot of columns that have blank spaces which represent no value in this case.
They were not identified as null values by python as they are empty spaces that are read as character values by
python. We will replace those values by NaNs and repeat the analysis

Before we do that we will replace the blank values from the column "MAILCODE" by "A" which would mean the address is okay (pl check the definition of the variable in the description)

In [ ]:
data['MAILCODE'].value_counts()

In [ ]:
data['MAILCODE'].unique()

In [ ]:
data['MAILCODE'] = data['MAILCODE'].apply(lambda x: x.replace(" ", "A"))
#data['MAILCODE'] = np.where(data['MAILCODE'] == " ", "A",data['MAILCODE'] )
#data['MAILCODE'] = data['MAILCODE'].apply(lambda x: x if (x != " ") else "A")
# function(arg1)
#df[col] = list(map(function,df[col]))
# function(arg1, arg2, arg3)
#df[col] = list(map(function,df[col],[arg2]*len(df), [arg3]*len(df)))

In [ ]:
# Now we can replace the rest space characters with np.NaN

In [ ]:
data = data.apply(lambda x: x.replace(" ", np.NaN))

In [ ]:
data.head()

# Activity

Use the previous function to obtain the columns with null values greater than 0.25 and store those columns in a list called "drop_columns_list"

In [ ]:
drop_columns_list = remove_cols_with_na(data, thr=0.25)
print(drop_columns_list)

In [ ]:
# End of activity

We would again repeat the same exercise as the last time. We will discuss it with the team, manager, and/or other stakeholders to see which columns we need to retain here.

Like the last time we will keep the following columns. **wealth1, wealth2 along with these VETERANS, SOLIH**

Remove those column's names from the list of columns to drop.

In [ ]:
# list comprehension
my_list = []

for number in range(1,11):
    my_list.append(number)

my_list

# List comprehension syntax
# [value for item in list]
my_list = [ number for number in range(1,11) ]
my_list

my_list = []

for number in range(1,11):
    if ( number%2 == 0 ):
        my_list.append(number)

my_list

# List comprehension syntax with a conditional
# [value for item in list if (condition)]
my_list = [ number for number in range(1,11) if (number%2 == 0)]
my_list

my_list = []

for number in range(1,11):
    if ( number%2 == 0 ):
        my_list.append(number)
    else:
        my_list.append(0)

my_list

# List comprehension syntax with a conditional with an else
# [value if (condition) else other_value for item in list]
my_list = [ number if (number%2 == 0) else 0 for number in range(1,11) ]
my_list

In [ ]:
def remove_columns(df, thr = 0.25, blacklist=None):
    ...
    if ( blacklist != None):
        drop_columns_list = [ col for col in drop_columns_list if col not in blacklist ]

In [ ]:
#creating an exception, drop columns list except the listed weatlh1...
drop_columns_list = [ col for col in drop_columns_list if col not in ['WEALTH1','WEALTH2','VETERANS','SOLIH'] ]

In [ ]:
len(drop_columns_list)

# X-y split

Since we have a huge number of features, it would be easier to work independently on numerical features and categorical features.

For the target variables, for now we will retain them both together. But later, we will build a classification model first where we would need the column TARGET_B only.

In [ ]:
Y = data[['TARGET_B', 'TARGET_D']]
Y.head()

In [ ]:
numerical = data.select_dtypes(np.number)
numerical = numerical.drop(columns = ['TARGET_B', 'TARGET_D'])
numerical.head()

In [ ]:
numerical.shape

In [ ]:
categorical = data.select_dtypes([object])
categorical.head()

In [ ]:
categorical.shape

# Working with categorical columns

We will work with the categorical features first. Look at the columns one by one. Some of the operations which we will perform are:

- Replace null values with the most occurring categories
- Reduce the number of categories in a column by grouping

It is important to note that some columns are defined by python as categorical/object types. There might be other columns defined as numerical that we want as categorical. We will look them later when we were working on numerical types.

In [ ]:
categorical.isna().sum()

# Categorical Variables

Here we will try to reduce the number of categories. An ideal way would have been to group the states into
regions. But in this case we will group all the states with counts less than 2500 into one category "other"

In [ ]:
df = pd.DataFrame(categorical['STATE'].value_counts()).reset_index()


df.columns = ['state', 'count']
other_states = list(df[df['count']<2500]['state'])

# other_states = ['CA','AR','MD']
def clean_state(x):
    if x in other_states:
        return 'other'
    else:
        return x

# clean_states(x,other_states) for x, other_states in zip(categorical['STATE'].values,other_states)
categorical['STATE'] = list(map(clean_state, categorical['STATE']))
#categorical['STATE'] = categorical['STATE'].apply(clean_state)

In [ ]:
categorical['STATE'].value_counts()

# 7.01 Lab | Revisiting Machine Learning Case Study

Lab instructions can be found [here](https://github.com/isg75/lab-revisiting-machine-learning/tree/master)

In [2]:
import pandas as pd

In [3]:
data = pd.read_csv('learningSet.csv')

/var/folders/fl/bzglcprn7zxd2fbfsh4kg_xm0000gr/T/ipykernel_89812/3673836359.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('learningSet.csv')


In [4]:
#reminder
def remove_cols_with_na(df: pd.DataFrame, thr = 0.25) -> list:
    nulls_percent_df = pd.DataFrame(df. isna().sum()/len(data)) .reset_index()
    nulls_percent_df.columns = ['column_name', 'nulls_percentage']
    return list(nulls_percent_df [nulls_percent_df['nulls_percentage'] > thr]['column_name'].values)

In [5]:
null_values = data.isnull().sum() # checking for null values in all columns

In [6]:
drop_list = []
drop_list.extend(['OSOURCE', 'ZIP']) #creating empty list drop_list and include outsource and zip

In [7]:
null_values

ODATEDW       0
OSOURCE       0
TCODE         0
STATE         0
ZIP           0
           ... 
MDMAUD_R      0
MDMAUD_F      0
MDMAUD_A      0
CLUSTER2    132
GEOCODE2    132
Length: 481, dtype: int64

In [8]:
columns_over_85_percent_nulls = null_values[null_values / len(data) > 0.85].index
display(columns_over_85_percent_nulls)

Index(['NUMCHLD', 'RDATE_3', 'RDATE_4', 'RDATE_5', 'RDATE_6', 'RDATE_7',
       'RDATE_10', 'RDATE_13', 'RDATE_15', 'RDATE_17', 'RDATE_20', 'RDATE_21',
       'RDATE_23', 'RAMNT_3', 'RAMNT_4', 'RAMNT_5', 'RAMNT_6', 'RAMNT_7',
       'RAMNT_10', 'RAMNT_13', 'RAMNT_15', 'RAMNT_17', 'RAMNT_20', 'RAMNT_21',
       'RAMNT_23'],
      dtype='object')

In [9]:
columns_over_85_percent_nulls

Index(['NUMCHLD', 'RDATE_3', 'RDATE_4', 'RDATE_5', 'RDATE_6', 'RDATE_7',
       'RDATE_10', 'RDATE_13', 'RDATE_15', 'RDATE_17', 'RDATE_20', 'RDATE_21',
       'RDATE_23', 'RAMNT_3', 'RAMNT_4', 'RAMNT_5', 'RAMNT_6', 'RAMNT_7',
       'RAMNT_10', 'RAMNT_13', 'RAMNT_15', 'RAMNT_17', 'RAMNT_20', 'RAMNT_21',
       'RAMNT_23'],
      dtype='object')

In [10]:
columns_over_85_percent_nulls = null_values[null_values / len(data) > 0.85].reset_index()
columns_over_85_percent_nulls

,index,0
0,NUMCHLD,83026
1,RDATE_3,95170
2,RDATE_4,95131
3,RDATE_5,95403
4,RDATE_6,94636
5,RDATE_7,86517
6,RDATE_10,84951
7,RDATE_13,83162
8,RDATE_15,88150
9,RDATE_17,86011


In [11]:
drop_list.extend(columns_over_85_percent_nulls['index'].values) #identifying columns above 85% missing values to add on drop list


,index,0
0,NUMCHLD,83026
1,RDATE_3,95170
2,RDATE_4,95131
3,RDATE_5,95403
4,RDATE_6,94636
5,RDATE_7,86517
6,RDATE_10,84951
7,RDATE_13,83162
8,RDATE_15,88150
9,RDATE_17,86011


In [12]:
drop_list

['OSOURCE',
 'ZIP',
 'NUMCHLD',
 'RDATE_3',
 'RDATE_4',
 'RDATE_5',
 'RDATE_6',
 'RDATE_7',
 'RDATE_10',
 'RDATE_13',
 'RDATE_15',
 'RDATE_17',
 'RDATE_20',
 'RDATE_21',
 'RDATE_23',
 'RAMNT_3',
 'RAMNT_4',
 'RAMNT_5',
 'RAMNT_6',
 'RAMNT_7',
 'RAMNT_10',
 'RAMNT_13',
 'RAMNT_15',
 'RAMNT_17',
 'RAMNT_20',
 'RAMNT_21',
 'RAMNT_23']

In [ ]:
drop_list = remove_cols_with_na(data, thr=0.85)

In [ ]:
def drop_list_85_na(df: pd.DataFrame, thr = 0.85) -> list:
    columns_over_85_percent_nulls = pd.DataFrame(data. isna().sum()/len(data)) .reset_index()
    columns_over_85_percent_nulls = ['column_name', 'nulls_percentage']
    return list(columns_over_85_percent_nulls [columns_over_85_percent_nulls['nulls_percentage'] > thr]['column_name'].values)

In [ ]:
columns_over_85_percent_nulls.head()

In [ ]:
data.head()

In [ ]:
gender_mapping = {'M': 'M', 'F': 'F', 'other': 'other'}

In [ ]:
data['GENDER'] = data['GENDER'].map(gender_mapping).fillna('other') #mapping GENDER column 

In [ ]:
display(data.head())

# 7.02

We will work on the column DOMAIN. Let's replace the null values with the category that is represented the most and then we will split the data into two columns DOMAIN_A and DOMAIN_B.

- DOMAIN_A will consist of the first character from the DOMAIN
- DOMAIN_B will consist of the second character from the DOMAIN

Then we will drop the original column DOMAIN

In [ ]:
categorical['DOMAIN'].value_counts(dropna=False)

In [ ]:
m = categorical[categorical['DOMAIN'].notna()]['DOMAIN'].mode()[0]
m

In [ ]:
categorical['DOMAIN'] = categorical['DOMAIN'].fillna(m)

Now let's create the new columns 'DOMAIN_A' and 'DOMAIN_B' with the first and second letter of column "DOMAIN" respectivelly

In [ ]:
categorical['DOMAIN_A'] = list(map(lambda x: x[0], categorical['DOMAIN']))
categorical['DOMAIN_B'] = list(map(lambda x: x[1], categorical['DOMAIN']))

Now we drop the original column

In [ ]:
categorical = categorical.drop(columns=['DOMAIN'])

In [ ]:
categorical.head()

# Activity

Take a look at all the columns in drop_columns_list (done on the class). These are the columns that we want to drop from the dataframe.

However, before removing all these columns, you discussed with your manager if there is any column that would still be important to keep in the analysis. Based on his subject matter expertise you were told that the WEALTH1 and WEALTH2 columns are important and should be kept.

On the other hand,  we would like to add the columns RDATE3, RAMNT_3 to the list of columns to drop because even although they are important, they have too many null values.

In [ ]:
# Your code here

In [ ]:
# End of the activity

In [ ]:
categorical.head()

MDMAUD_XXX are also like MDMAUD



In [ ]:
categorical['MDMAUD_R'].value_counts()

In [ ]:
#drop_columns_list = drop_columns_list + ['MDMAUD_R', 'MDMAUD_F','MDMAUD_A']
drop_columns_list += ['MDMAUD_R', 'MDMAUD_F','MDMAUD_A']

Replacing NULL values

In [ ]:
categorical['CLUSTER'].dtype

In [ ]:
categorical['CLUSTER'].value_counts(dropna=False)

In [ ]:
categorical[['CLUSTER']].head()

In [ ]:
categorical['CLUSTER'][0]

In [ ]:
m = categorical[categorical['CLUSTER'].notna()]['CLUSTER'].mode()[0]
m

In [ ]:
categorical['CLUSTER'] = categorical['CLUSTER'].fillna(m)

In [ ]:
categorical['HOMEOWNR'].value_counts(dropna=False)

In [ ]:
m = categorical[categorical['HOMEOWNR'].notna()]['HOMEOWNR'].mode()[0]

In [ ]:
categorical['HOMEOWNR'] = categorical['HOMEOWNR'].fillna(m)

In [ ]:
#categorical['GENDER'].value_counts(dropna=False)

In [ ]:
categorical.columns

In [ ]:
list(categorical.columns).index("GENDER")

In [ ]:
#def clean_gender(x):
#    if ( x not in ['F','M'] and np.isna(x)==False):
#        return "other"
#    else:
#        return x

In [ ]:
#ml = list(categorical[categorica['GENDER].isin(['U','J','C','A])].index)
ml =list(categorical[(categorical['GENDER'].isin(['F','M']) == False) & (categorical['GENDER'].notna())]['GENDER'].index)
categorical.iloc[ml,list(categorical.columns).index("GENDER")] = "other"

In [ ]:
categorical['GENDER'].value_counts(dropna=False)

In [ ]:
categorical.head()

Let's drop the rows with NA's in gender.

In [ ]:
ml = list(categorical[categorical['GENDER'].notna()].index)
categorical = categorical.loc[ml,:]
numerical = numerical.loc[ml,:]

Dealing with redundant information

In [ ]:
categorical['RFA_2'].value_counts(dropna=False)

We will remove all the columns "RFA_2X" except ['RFA_2R','RFA_2A']

In [ ]:
drop_columns_list += [col for col in categorical.columns if (("RFA" in col) and ( col not in ['RFA_2R','RFA_2A']))]

In [ ]:
drop_columns_list

In [ ]:
list(categorical.columns)

In [ ]:
cat_columns_to_drop = [col for col in drop_columns_list if col in list(categorical.columns)]
cat_columns_to_drop

In [ ]:
categorical = categorical.drop(columns=cat_columns_to_drop, axis=1)
categorical.head()

In [ ]:
categorical.isna().sum()

In [ ]:
categorical.isna().sum()/len(categorical)

In [ ]:
categorical.drop(columns=['SOLIH','VETERANS'],axis = 1 , inplace = True)

In [ ]:
categorical.shape

# Working with numerical columns

In [ ]:
numerical.head()

In [ ]:
numerical.shape

In [ ]:
df = pd.DataFrame(numerical.isna().sum()).reset_index()
df.columns = ['column_name', 'nulls']
df[df['nulls']>0]

# Activity

Determine wich numerical columns have a percentage of NA's above 25%.

In [ ]:
# Your code here

In [ ]:
# End of activity

In [ ]:
np.median(numerical[numerical['AGE'].notna()]['AGE'])
#numerical[numerical['AGE'].notna()]['AGE'].median()

In [ ]:
np.mean(numerical[numerical['AGE'].notna()]['AGE'])

# Exploring the effect of several missing values imputation techniques

In [ ]:
#numerical['AGE'] = numerical["AGE"].fillna(np.mean(numerical['AGE']))
fig, ax = plt.subplots(1,3,figsize=(16,5))
sns.histplot(numerical['AGE'], ax = ax[0])
sns.histplot(numerical["AGE"].fillna(np.mean(numerical[numerical['AGE'].notna()]['AGE'])), ax = ax[1])
sns.histplot(numerical['AGE'].fillna(np.median(numerical[numerical['AGE'].notna()]['AGE'])),ax = ax[2])
plt.show()
numerical['AGE'] = numerical["AGE"].fillna(np.median(numerical['AGE']))

In [ ]:
#indexes_with_na = numerical[numerical['AGE'].isna()].index
#for index in indexes_with_na:
#    x = np.random() # [ 0,1]
#    numerical.iloc[index,list(numerical.columns).index('AGE')] = np.quantile(numerical['AGE'],x)# kde

In [ ]:
#sns.distplot(numerical['INCOME'])  # this will not work as there are NaNs in the column
sns.histplot(numerical[numerical['INCOME'].notna()]['INCOME'])
plt.show()

In [ ]:
# looks like the variable is actually categorical. We can verify it by using value_counts()
print(numerical['INCOME'].value_counts(dropna=False))
#numerical['INCOME'] = numerical['INCOME'].astype('object')

In [ ]:
m = numerical['INCOME'].mode()[0]
numerical['INCOME'] = numerical['INCOME'].fillna(m) # Replacing the null values with the most represented categoty

In [ ]:
#numerical[numerical['CLUSTER2'].notna()]
sns.histplot(numerical[numerical['CLUSTER2'].isna()==False]['CLUSTER2'],kde=True)
plt.show()

In [ ]:
numerical['CLUSTER2'] = numerical['CLUSTER2'].fillna(np.ceil(np.mean(numerical['CLUSTER2'])))

# Activity

Drop all the numerical columns that contain the string "ADATE_" in the column name. We are assuming that the date when the previous mail was done is not significant in the respondents decision to give donation.

In [ ]:
# Your code here

In [ ]:
# End of activity

#Lab 2

Lab instructions can be found [here](https://github.com/ironhack-labs/lab-feature-engineering)